# Weather Feature
Die Wetterdaten aus der Radverkehrszählung der Standmünster werden geladen und in ein passendes Format überführt.

In [1]:
import pandas as pd
import numpy as np
import pickle
import datetime

In [2]:
with open('df_linie1_hin.pickle', 'rb') as fp:
    df = pickle.load(fp)
df.head()

,Datum,Wochentag,Fahrweg,Fahrt,Fahrzeugnummer,Linienverlauf,SollAb,IstAb,GPS-Breite Soll,GPS-Länge Soll,Verspätung,Verspätung_total_seconds
0,2016-10-10,Montag,2,1081890,5556,30 Ludgeriplatz B,2016-10-10 05:17:00,2016-10-10 05:17:58,519558997.0,76278508.0,00:00:58,58.0
1,2016-10-10,Montag,2,1081890,5556,31 Goebenstr.,2016-10-10 05:19:00,2016-10-10 05:19:03,51952888.0,76254844.0,00:00:03,3.0
2,2016-10-10,Montag,2,1081890,5556,32 St. Joseph-Kirche,2016-10-10 05:20:00,2016-10-10 05:20:00,519493516.0,76243561.0,00:00:00,0.0
3,2016-10-10,Montag,2,1081890,5556,33 Alter Sch¿tzenhof A,2016-10-10 05:21:00,2016-10-10 05:20:44,519460313.0,76232794.0,-1 days +23:59:44,-16.0
4,2016-10-10,Montag,2,1081890,5556,34 Metzer Str.,2016-10-10 05:23:00,2016-10-10 05:23:20,519422225.0,76234541.0,00:00:20,20.0


Data can be downloaded from http://www.stadt-muenster.de/verkehrsplanung/verkehr-in-zahlen/radverkehrszaehlungen/wolbecker-strasse.html


In [3]:
# Load bicyle data that contains wheather information.
df_bicyle = pd.read_csv('zaehlstelle_wolbecker_2016_stundenauswertung.csv', skipfooter=2, engine='python')
df_bicyle.head()

,Stunden,Wolbecker Straße (gesamt),FR stadteinwärts,FR stadtauswärts,Temperatur (°C),Windstärke (km/h),Wetter
0,01/01/2016 00:00,89,30.0,59.0,3.0,9.0,Klarer Himmel
1,01/01/2016 01:00,164,73.0,91.0,3.0,7.0,Klarer Himmel
2,01/01/2016 02:00,188,101.0,87.0,3.0,9.0,Klarer Himmel
3,01/01/2016 03:00,186,103.0,83.0,3.0,9.0,teilweise bewölkt
4,01/01/2016 04:00,141,82.0,59.0,2.0,6.0,wolkig


In [4]:
# Parse to timestamps.
df_bicyle['Stunden'] = pd.to_datetime(df_bicyle['Stunden'], format='%d/%m/%Y %H:%M')
df_bicyle.dtypes

Stunden                      datetime64[ns]
Wolbecker Straße (gesamt)            object
FR stadteinwärts                    float64
FR stadtauswärts                    float64
Temperatur (°C)                     float64
Windstärke (km/h)                   float64
Wetter                               object
dtype: object

In [5]:
# Interpolate missing values using the value with forward fill.
df_bicyle['Wetter'] = df_bicyle['Wetter'].fillna(method='ffill')
df_bicyle['Temperatur (°C)'] = df_bicyle['Temperatur (°C)'].fillna(df_bicyle['Temperatur (°C)'].mean())
df_bicyle['Windstärke (km/h)'] = df_bicyle['Windstärke (km/h)'].fillna(df_bicyle['Windstärke (km/h)'].mean())
df_bicyle.head()

,Stunden,Wolbecker Straße (gesamt),FR stadteinwärts,FR stadtauswärts,Temperatur (°C),Windstärke (km/h),Wetter
0,2016-01-01 00:00:00,89,30.0,59.0,3.0,9.0,Klarer Himmel
1,2016-01-01 01:00:00,164,73.0,91.0,3.0,7.0,Klarer Himmel
2,2016-01-01 02:00:00,188,101.0,87.0,3.0,9.0,Klarer Himmel
3,2016-01-01 03:00:00,186,103.0,83.0,3.0,9.0,teilweise bewölkt
4,2016-01-01 04:00:00,141,82.0,59.0,2.0,6.0,wolkig


In [6]:
# Fake 2017 data, since they are missing.
df_bicyle2017 = df_bicyle.copy()
df_bicyle2017['Stunden'] = df_bicyle2017['Stunden'] + datetime.timedelta(days=366)
df_bicyle2017.head()

,Stunden,Wolbecker Straße (gesamt),FR stadteinwärts,FR stadtauswärts,Temperatur (°C),Windstärke (km/h),Wetter
0,2017-01-01 00:00:00,89,30.0,59.0,3.0,9.0,Klarer Himmel
1,2017-01-01 01:00:00,164,73.0,91.0,3.0,7.0,Klarer Himmel
2,2017-01-01 02:00:00,188,101.0,87.0,3.0,9.0,Klarer Himmel
3,2017-01-01 03:00:00,186,103.0,83.0,3.0,9.0,teilweise bewölkt
4,2017-01-01 04:00:00,141,82.0,59.0,2.0,6.0,wolkig


In [7]:
# Concat dataframes of 2017 and 2018.
df_bicyle = pd.concat([df_bicyle, df_bicyle2017])
df_bicyle.head()

,Stunden,Wolbecker Straße (gesamt),FR stadteinwärts,FR stadtauswärts,Temperatur (°C),Windstärke (km/h),Wetter
0,2016-01-01 00:00:00,89,30.0,59.0,3.0,9.0,Klarer Himmel
1,2016-01-01 01:00:00,164,73.0,91.0,3.0,7.0,Klarer Himmel
2,2016-01-01 02:00:00,188,101.0,87.0,3.0,9.0,Klarer Himmel
3,2016-01-01 03:00:00,186,103.0,83.0,3.0,9.0,teilweise bewölkt
4,2016-01-01 04:00:00,141,82.0,59.0,2.0,6.0,wolkig


In [8]:
df_bicyle = df_bicyle.reset_index(drop=True)
df_bicyle.head()

,Stunden,Wolbecker Straße (gesamt),FR stadteinwärts,FR stadtauswärts,Temperatur (°C),Windstärke (km/h),Wetter
0,2016-01-01 00:00:00,89,30.0,59.0,3.0,9.0,Klarer Himmel
1,2016-01-01 01:00:00,164,73.0,91.0,3.0,7.0,Klarer Himmel
2,2016-01-01 02:00:00,188,101.0,87.0,3.0,9.0,Klarer Himmel
3,2016-01-01 03:00:00,186,103.0,83.0,3.0,9.0,teilweise bewölkt
4,2016-01-01 04:00:00,141,82.0,59.0,2.0,6.0,wolkig


In [9]:
# Drop useless columns.
df_bicyle = df_bicyle.drop(columns=['Wolbecker Straße (gesamt)', 'FR stadteinwärts', 'FR stadtauswärts'])

In [10]:
# Save results.
df_bicyle.to_pickle('df_weather.pickle')